In [1]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from operator import itemgetter
import time
from selenium import webdriver
from selenium.webdriver.common.by import By


In [ ]:
school_names = ['Northwest', 'North Platte', 'Gothenburg']
state = 'Nebraska'

In [2]:
headers = ['school_name', 'mascot_city', 'state_abbr', 'sport', 'division', 'school_id']
school_queries = [(state, name_) for name_ in school_names]


def get_school_id(query): # query (state, team)
    state = query[0].capitalize()
    name = query[1].lower()
    
    url = f"https://www.athletic.net/CrossCountry/{state}/"
    response = requests.get(url) 
    soup = BeautifulSoup(response.content, 'html.parser')
    refs = soup.find_all("a", href=True)
    school_refs = [ref for ref in refs if '/team/' in ref['href']]
    schools = soup.find_all("span", class_="truncate")
    
    for i, j in zip(schools, school_refs):
        school_name = i.text.lower()
        
        if school_name == name:
            return re.search(r"/(\d+)/", j['href']).group(1)
    
def getSchools():
    return [get_school_id(s) for s in school_queries]

def schoolTopTimes(schoolID): 
    data = []
    url = "https://www.athletic.net/CrossCountry/seasonbest?SchoolID=" + str(schoolID)
    r = requests.get(url)   
    soup = BeautifulSoup(r.content, 'html.parser')
    txt = soup.get_text()

    _,txt = txt.split("5,000")

    x =  re.findall("([0-9])\.([0-9][0-9]?)(.*?\s.*?)([1-9][0-9]\:[0-9][0-9](?:\.?\d))",txt)
    c = 1

    for item in x:
        row = [item[0], item[1], item[2], item[3], schoolID]
        
        if int(row[0]) != c or int(row[0]) == None:
            c+=1
            continue
            
        else:
            c+=1
            data.append(row)

    return data



def main():
    timesList = []
    # list of schools at the meet
    schoolIDList = getSchools()

    # for each school at the meet, get the top times
    for schoolID in schoolIDList:
        schoolDataList = schoolTopTimes(schoolID)

        # add the times to the list
        for time in schoolDataList:
            timesList.append(time)

    # sort the list by time
    timesList = sorted(timesList, key=itemgetter(3))
    count = 1
    for x in timesList:
        print(f'{count}. {x[3]} {x[2]}')
        count+=1


if __name__ == "__main__":
    main()
    

1. 15:53.2 Tyler Hetz
2. 15:58.5 Ethan Smith
3. 16:27.0 Parker Graves
4. 16:43.7 Aiden Hawks
5. 16:43.9 Ethan Olsen
6. 16:44.8 Luke Tegtmeier
7. 16:47.4 Jack Knapp
8. 16:49.9 Rian Teets
9. 16:56.5 Yahriel Gaeta
10. 17:08.2 Nathan Sager
11. 17:09.0 Caden Sheffield
12. 17:36.3 Evan Pelzer
13. 17:48.0 Christian Mendoza
14. 18:11.4 Keith Krolikowski
15. 18:17.3 Ethan Gydesen
16. 18:22.2 Noah Short
17. 18:25.0 James Peters
18. 18:26.8 Joe Sutherland
19. 18:39.1 Lucas Therrien
20. 18:51.7 Charlie Short
21. 19:00.6 Aidan Shavlik
22. 19:01.6 Khale Lindiman
23. 19:08.8 Nolan Teets
24. 19:12.7 Kent Hansen
25. 19:12.8 Ty Kreis
26. 19:16.0 Luke Thramer
27. 20:16.5 Chris Flores
